# Clase 27/02

https://r2022.netlify.app/

In [1]:
install.packages("readxl")
library(readxl)
install.packages("readr")
library(readr)
install.packages("stringr")
library(stringr)
install.packages("dplyr")
library(dplyr)
source("/work/AiRoute/university-stuff/statistical_computing/sources/mismacros.txt")
install.packages("sqldf")
library(sqldf)
install.packages("dplyr")
library(dplyr)

Installing package into ‘/work/.R/library’
(as ‘lib’ is unspecified)

Warning message in install.packages("readxl"):
“installation of package ‘readxl’ had non-zero exit status”
Installing package into ‘/work/.R/library’
(as ‘lib’ is unspecified)

Warning message in install.packages("readr"):
“installation of package ‘readr’ had non-zero exit status”
Installing package into ‘/work/.R/library’
(as ‘lib’ is unspecified)

Installing package into ‘/work/.R/library’
(as ‘lib’ is unspecified)

Warning message in install.packages("dplyr"):
“installation of package ‘dplyr’ had non-zero exit status”

Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



table  :  Month
weights: 1
                  Frequency
   | ----------------------
5  |                     31
6  |                     30
7  |                     31
8  |                     31
9  |  

In [2]:
Municipios <- readRDS("/work/AiRoute/university-stuff/statistical_computing/sources/outputs/Municipios.RDS")

In [3]:
Estaciones <- read_excel("/work/AiRoute/university-stuff/statistical_computing/sources/Estaciones.xlsx")
Estaciones <- within(Estaciones,Depmun <- str_sub(Depmun,1,5))

## DPLYR

### Cual es la región con el mayor número de departamentos?

In [4]:
Municipios %>% 
group_by(Region) %>%
summarise(Deps=n_distinct(Dep)) %>%
arrange(desc(Deps)) %>%
head(n=1)

Region,Deps
<chr>,<int>
Región Caribe,8


### Ejemplos varios

Conjunto de datos con todos los departamentos con al menos 650 mil habitantes y al menos 10 mil km2
 de superficie, ordenados de mayor a menor con relación a su densidad poblacional, y que incluye la siguiente información: número de municipios, superficie, población y densidad poblacional.

In [5]:
e10 <- Municipios %>%
       group_by(Dep,Departamento) %>%
       summarise(nmunicipios=n(),totpob=sum(Poblacion),totsup=sum(Superficie),
                 denspob=totpob/totsup) %>%
       filter(totpob >= 650000 & totsup >= 10000) %>%
       arrange(desc(denspob)) %>% 
       as.data.frame()

#all.equal(e10,E10,check.attributes=FALSE)
str(e10)

'data.frame':	17 obs. of  6 variables:
 $ Dep         : chr  "76" "25" "05" "70" ...
 $ Departamento: chr  "Valle Del Cauca" "Cundinamarca" "Antioquia" "Sucre" ...
 $ nmunicipios : int  42 116 125 26 45 87 40 28 37 30 ...
 $ totpob      : num  4506768 3085522 6550206 928984 2120925 ...
 $ totsup      : num  20666 22370 62805 10592 26303 ...
 $ denspob     : num  218.1 137.9 104.3 87.7 80.6 ...


Conjunto de datos con todos los departamentos con al menos 650 mil habitantes y al menos 10 mil de superficie, ordenados de mayor a menor con relación a su densidad poblacional, y que incluye la siguiente información: número de municipios, superficie, población, y densidad poblacional, todo referente a municipios con índice de ruralidad mayor a 40.

In [6]:
e11 <- Municipios %>%
        filter(Irural > 40) %>%# "WHERE" de sql
        group_by(Dep,Departamento) %>%
        summarise(nmunicipios=n(),totpob=sum(Poblacion),totsup=sum(Superficie),
                  denspob=totpob/totsup) %>%
        filter(totpob >= 650000 & totsup >= 10000) %>%#"HAVING" de sql
        arrange(desc(denspob)) %>%
        as.data.frame()

#all.equal(e11,E11,check.attributes=FALSE)
str(e11)

'data.frame':	7 obs. of  6 variables:
 $ Dep         : chr  "44" "23" "20" "52" ...
 $ Departamento: chr  "La Guajira" "Córdoba" "Cesar" "Nariño" ...
 $ nmunicipios : int  15 18 23 45 34 86 30
 $ totpob      : num  927506 813722 734244 757725 653793 ...
 $ totsup      : num  20622 18778 18304 25475 22520 ...
 $ denspob     : num  45 43.3 40.1 29.7 29 ...


Existen dos o más municipios con el mismo nombre? Cuantos? Cuales? En qué departamentos están ubicados?

In [7]:
original <- Sys.getlocale("LC_COLLATE")#Arreglar la oredancion de la filas (no del todo verdad)
Sys.setlocale("LC_COLLATE", "C")

Repetidos <- Municipios %>%
             group_by(Municipio) %>%
             summarise(veces = n()) %>%
             filter(veces > 1) %>%
             arrange(desc(veces)) %>%
             as.data.frame()

str(Repetidos) 
head(Repetidos)

e12 <- Municipios %>%
       select(Municipio,Departamento,Dep,Depmun) %>%
       filter(Municipio %in% with(Repetidos,Municipio)) %>%#with(Arreglo, Columna) ---> Repetidos$Municipio
       arrange(Municipio,Departamento) %>%
       as.data.frame()

Sys.setlocale("LC_COLLATE", original)

#all.equal(e12,E12,check.attributes=FALSE)
str(e12)
head(e12)
tail(e12)

[1] "C"

'data.frame':	69 obs. of  2 variables:
 $ Municipio: chr  "La Unión" "Villanueva" "Albania" "Argelia" ...
 $ veces    : int  4 4 3 3 3 3 3 3 3 3 ...


,Municipio,veces
,<chr>,<int>
1,La Unión,4
2,Villanueva,4
3,Albania,3
4,Argelia,3
5,Bolívar,3
6,Buenavista,3


[1] "C"

'data.frame':	157 obs. of  4 variables:
 $ Municipio   : chr  "Albania" "Albania" "Albania" "Albán" ...
 $ Departamento: chr  "Caquetá" "La Guajira" "Santander" "Cundinamarca" ...
 $ Dep         : chr  "18" "44" "68" "25" ...
 $ Depmun      : chr  "18029" "44035" "68020" "25019" ...
 - attr(*, "na.action")= 'omit' Named int 1098
  ..- attr(*, "names")= chr "1098"


,Municipio,Departamento,Dep,Depmun
,<chr>,<chr>,<chr>,<chr>
1,Albania,Caquetá,18,18029
2,Albania,La Guajira,44,44035
3,Albania,Santander,68,68020
4,Albán,Cundinamarca,25,25019
5,Albán,Nariño,52,52019
6,Argelia,Antioquia,05,05055


,Municipio,Departamento,Dep,Depmun
,<chr>,<chr>,<chr>,<chr>
152,Venecia,Antioquia,05,05861
153,Venecia,Cundinamarca,25,25506
154,Villanueva,Bolívar,13,13873
155,Villanueva,Casanare,85,85440
156,Villanueva,La Guajira,44,44874
157,Villanueva,Santander,68,68872


Existen dos o más municipios con el mismo nombre que hacen parte de la misma región? Cuantos? Cuales?

In [8]:
original <- Sys.getlocale("LC_COLLATE")
Sys.setlocale("LC_COLLATE", "C")

Repetidos <- Municipios %>%
             group_by(Region,Municipio) %>%
             summarise(veces = n()) %>%
             filter(veces > 1) %>%
             arrange(desc(veces)) %>%
             as.data.frame()

str(Repetidos) 

e13 <- Municipios %>%
       select(Municipio,Region,Departamento,Dep,Depmun) %>%
       mutate(newvar = str_c(Region, Municipio)) %>%
       filter(newvar %in% with(Repetidos,newvar)) %>%
       arrange(Municipio,Region) %>%
       as.data.frame()

Sys.setlocale("LC_COLLATE", original)

#all.equal(e13,E13,check.attributes=FALSE)
str(e13)

[1] "C"

'data.frame':	11 obs. of  3 variables:
 $ Region   : chr  "Región Caribe" "Región Caribe" "Región Caribe" "Región Centro Oriente" ...
 $ Municipio: chr  "Buenavista" "Manaure" "Villanueva" "Cabrera" ...
 $ veces    : int  2 2 2 2 2 2 2 2 2 2 ...


[1] "C"

'data.frame':	1117 obs. of  6 variables:
 $ Municipio   : chr  "Abejorral" "Abrego" "Abriaquí" "Acacias" ...
 $ Region      : chr  "Región Eje Cafetero" "Región Centro Oriente" "Región Eje Cafetero" "Región Llano" ...
 $ Departamento: chr  "Antioquia" "Norte De Santander" "Antioquia" "Meta" ...
 $ Dep         : chr  "05" "54" "05" "50" ...
 $ Depmun      : chr  "05002" "54003" "05004" "50006" ...
 $ newvar      : chr  "Región Eje CafeteroAbejorral" "Región Centro OrienteAbrego" "Región Eje CafeteroAbriaquí" "Región LlanoAcacias" ...
 - attr(*, "na.action")= 'omit' Named int 1098
  ..- attr(*, "names")= chr "1098"


In [9]:
original <- Sys.getlocale("LC_COLLATE")
Sys.setlocale("LC_COLLATE", "C")

Repetidos <- Municipios %>%
             group_by(Region,Municipio) %>%
             summarise(veces = n()) %>%
             filter(veces > 1) %>%
             arrange(desc(veces)) %>%
             as.data.frame()

#str(Repetidos) 

e13a <- Municipios %>%
       select(Municipio,Region,Departamento,Dep,Depmun) %>%
       filter(str_c(Region,Municipio) %in% with(Repetidos,str_c(Region,Municipio))) %>%
       arrange(Municipio,Region) %>%
       as.data.frame()

Sys.setlocale("LC_COLLATE", original)

all.equal(e13, e13a,check.attributes=FALSE)
#str(e13a)

[1] "C"

[1] "C"

[1] "Length mismatch: comparison on first 5 components"                               
 [2] "Component “Municipio”: Lengths (1117, 22) differ (string compare on first 22)"   
 [3] "Component “Municipio”: 22 string mismatches"                                     
 [4] "Component “Region”: Lengths (1117, 22) differ (string compare on first 22)"      
 [5] "Component “Region”: 16 string mismatches"                                        
 [6] "Component “Departamento”: Lengths (1117, 22) differ (string compare on first 22)"
 [7] "Component “Departamento”: 19 string mismatches"                                  
 [8] "Component “Dep”: Lengths (1117, 22) differ (string compare on first 22)"         
 [9] "Component “Dep”: 19 string mismatches"                                           
[10] "Component “Depmun”: Lengths (1117, 22) differ (string compare on first 22)"      
[11] "Component “Depmun”: 22 string mismatches"

Conjunto de datos con los municipios cuyos nombres inician con "A" o terminan con "o", no importa si llevan tilde o no.

In [10]:
e14 <- Municipios %>% 
       select(Dep,Departamento,Municipio,Depmun) %>% 
       filter(str_detect(Municipio,"^[AÁ]") | str_detect(Municipio,"[oó]$")) %>%
       as.data.frame()

e14a <- Municipios %>% 
        select(Dep,Departamento,Municipio, Depmun) %>% 
        filter(str_detect(Municipio,"^[AÁ]|[oó]$")) %>%
        as.data.frame()

all.equal(e14,e14a,check.attributes=FALSE)
#all.equal(e14,E14,check.attributes=FALSE)
str(e14) 
head(e14)

[1] TRUE

'data.frame':	308 obs. of  4 variables:
 $ Dep         : chr  "05" "05" "05" "05" ...
 $ Departamento: chr  "Antioquia" "Antioquia" "Antioquia" "Antioquia" ...
 $ Municipio   : chr  "Abejorral" "Abriaquí" "Alejandría" "Amagá" ...
 $ Depmun      : chr  "05002" "05004" "05021" "05030" ...
 - attr(*, "na.action")= 'omit' Named int 1098
  ..- attr(*, "names")= chr "1098"


,Dep,Departamento,Municipio,Depmun
,<chr>,<chr>,<chr>,<chr>
1,05,Antioquia,Abejorral,05002
2,05,Antioquia,Abriaquí,05004
3,05,Antioquia,Alejandría,05021
4,05,Antioquia,Amagá,05030
5,05,Antioquia,Amalfi,05031
6,05,Antioquia,Andes,05034


Conjunto de datos con los municipios cuyos nombres inician con "A" y terminan con "o", no importa si llevan tilde o no.

In [11]:
e15 <- Municipios %>% 
       select(Dep,Departamento,Municipio,Depmun) %>% 
       filter(str_detect(Municipio,"^[AÁ]") & str_detect(Municipio,"[oó]$")) %>%
       as.data.frame()

e15a <- Municipios %>% 
        select(Dep,Departamento,Municipio,Depmun) %>% 
        filter(str_detect(Municipio,"^[AÁ].*[oó]$")) %>%
        as.data.frame()

all.equal(e15,e15a,check.attributes=FALSE)
#all.equal(e15,E15,check.attributes=FALSE)
str(e15) 
head(e15)

[1] TRUE

'data.frame':	15 obs. of  4 variables:
 $ Dep         : chr  "05" "13" "13" "15" ...
 $ Departamento: chr  "Antioquia" "Bolívar" "Bolívar" "Boyacá" ...
 $ Municipio   : chr  "Apartadó" "Altos Del Rosario" "Arroyohondo" "Arcabuco" ...
 $ Depmun      : chr  "05045" "13030" "13062" "15051" ...
 - attr(*, "na.action")= 'omit' Named int 1098
  ..- attr(*, "names")= chr "1098"


,Dep,Departamento,Municipio,Depmun
,<chr>,<chr>,<chr>,<chr>
1,05,Antioquia,Apartadó,05045
2,13,Bolívar,Altos Del Rosario,13030
3,13,Bolívar,Arroyohondo,13062
4,15,Boyacá,Arcabuco,15051
5,25,Cundinamarca,Apulo,25599
6,27,Chocó,Alto Baudo,27025


Conjunto de datos con los municipios cuyo nombre tiene a la letra "e" en las posiciones 2 y 4, no importa si llevan tilde o no.

In [12]:
e16 <- Municipios %>% 
       select(Dep,Departamento,Municipio,Depmun) %>% 
       filter(str_sub(Municipio,2,2) %in% c("e","é") & str_sub(Municipio,4,4) %in% c("e","é")) %>%
       as.data.frame()

e16a <- Municipios %>% 
        select(Dep,Departamento,Municipio,Depmun) %>% 
        filter(str_detect(Municipio,"^.[eé].[eé]")) %>%
        as.data.frame()

all.equal(e16,e16a,check.attributes=FALSE)
#all.equal(e16,E16,check.attributes=FALSE)
str(e16)

[1] TRUE

'data.frame':	18 obs. of  4 variables:
 $ Dep         : chr  "05" "05" "05" "08" ...
 $ Departamento: chr  "Antioquia" "Antioquia" "Antioquia" "Atlántico" ...
 $ Municipio   : chr  "Medellín" "Remedios" "Venecia" "Repelón" ...
 $ Depmun      : chr  "05001" "05604" "05861" "08606" ...
 - attr(*, "na.action")= 'omit' Named int 1098
  ..- attr(*, "names")= chr "1098"


Conjunto de datos con los municipios cuyos nombres constan de más de una palabra.

In [13]:
e17 <- Municipios %>% 
       select(Dep,Departamento,Municipio,Depmun) %>% 
       filter(str_count(Municipio," ")>=1) %>%
       as.data.frame()

e17a <- Municipios %>% 
        select(Dep,Departamento,Municipio,Depmun) %>% 
        filter(str_detect(Municipio,".* .*")) %>%
        as.data.frame()

all.equal(e17,e17a,check.attributes=FALSE)
#all.equal(e17,E17,check.attributes=FALSE)
str(e17) 
head(e17)

[1] TRUE

'data.frame':	307 obs. of  4 variables:
 $ Dep         : chr  "05" "05" "05" "05" ...
 $ Departamento: chr  "Antioquia" "Antioquia" "Antioquia" "Antioquia" ...
 $ Municipio   : chr  "Santafé De Antioquia" "Ciudad Bolívar" "El Carmen De Viboral" "Don Matías" ...
 $ Depmun      : chr  "05042" "05101" "05148" "05237" ...
 - attr(*, "na.action")= 'omit' Named int 1098
  ..- attr(*, "names")= chr "1098"


,Dep,Departamento,Municipio,Depmun
,<chr>,<chr>,<chr>,<chr>
1,05,Antioquia,Santafé De Antioquia,05042
2,05,Antioquia,Ciudad Bolívar,05101
3,05,Antioquia,El Carmen De Viboral,05148
4,05,Antioquia,Don Matías,05237
5,05,Antioquia,El Bagre,05250
6,05,Antioquia,Gómez Plata,05310


### Ejemplos de Joins

Cuantos y cuales municipios tienen al menos una estación de monitoreo climático?

In [14]:
e18 <- Estaciones %>% 
       distinct(Depmun) %>% as.data.frame()

#all.equal(e18,E18,check.attributes=FALSE)
str(e18)

'data.frame':	806 obs. of  1 variable:
 $ Depmun: chr  "05209" "05237" "05250" "05148" ...


Conjunto de datos con todos los municipios del país, que incluye una variable que informa el número de estaciones de monitoreo climático con que cuenta cada municipio.

In [15]:
Emon <- Estaciones %>% 
        group_by(Depmun) %>%
        summarise(nem=n())

e19 <- Municipios %>%
       left_join(Emon,by=c("Depmun"="Depmun")) %>%
       mutate(nest=case_when(is.na(nem) ~ 0L,TRUE ~ nem)) %>%#OL -> Es un integer  | 0 -> es numeric
       select(everything(Municipios),nest) %>%
       as.data.frame()

#all.equal(e19,E19,check.attributes=FALSE)
str(e19)

'data.frame':	1117 obs. of  12 variables:
 $ Departamento: chr  "Antioquia" "Antioquia" "Antioquia" "Antioquia" ...
 $ Dep         : chr  "05" "05" "05" "05" ...
 $ Municipio   : chr  "Medellín" "Abejorral" "Abriaquí" "Alejandría" ...
 $ Depmun      : chr  "05001" "05002" "05004" "05021" ...
 $ Superficie  : num  374.8 507.1 297 128.9 84.1 ...
 $ Poblacion   : num  2483545 20258 2710 4669 30777 ...
 $ Irural      : num  5 45 58 48 28 54 38 37 47 58 ...
 $ Region      : chr  "Región Eje Cafetero" "Región Eje Cafetero" "Región Eje Cafetero" "Región Eje Cafetero" ...
 $ Tipo        : chr  "Capital" "Otro" "Otro" "Otro" ...
 $ Zona        : chr  "Urbano" "Rural" "Rural" "Rural" ...
 $ denspobl    : num  6625.78 39.95 9.13 36.21 365.82 ...
 $ nest        : int  10 2 1 0 0 3 3 0 2 3 ...
 - attr(*, "na.action")= 'omit' Named int 1098
  ..- attr(*, "names")= chr "1098"


Existen municipios que no tienen ninguna estación de monitoreo climático? Cuantos? Cuales?

In [16]:
e20 <- Municipios %>%
       left_join(Emon,by=c("Depmun"="Depmun")) %>%
       filter(is.na(nem)) %>% select(everything(Municipios))%>%
       arrange(Depmun) %>%
       as.data.frame()

e20a <- Municipios %>% arrange(Depmun) %>%
        filter(!(Depmun %in% with(Estaciones,Depmun))) %>%
        as.data.frame()

all.equal(e20,e20a,check.attributes=FALSE)
#all.equal(e20,E20,check.attributes=FALSE)

str(e20) 
head(e20)
tail(e20)

[1] TRUE

'data.frame':	311 obs. of  11 variables:
 $ Departamento: chr  "Antioquia" "Antioquia" "Antioquia" "Antioquia" ...
 $ Dep         : chr  "05" "05" "05" "05" ...
 $ Municipio   : chr  "Alejandría" "Amagá" "Angelópolis" "Argelia" ...
 $ Depmun      : chr  "05021" "05030" "05036" "05055" ...
 $ Superficie  : num  128.9 84.1 81.9 245.3 376.3 ...
 $ Poblacion   : num  4669 30777 5790 7577 8065 ...
 $ Irural      : num  48 28 37 46 52 46 34 20 57 42 ...
 $ Region      : chr  "Región Eje Cafetero" "Región Eje Cafetero" "Región Eje Cafetero" "Región Eje Cafetero" ...
 $ Tipo        : chr  "Otro" "Otro" "Otro" "Otro" ...
 $ Zona        : chr  "Rural" "Urbano" "Urbano" "Rural" ...
 $ denspobl    : num  36.2 365.8 70.7 30.9 21.4 ...
 - attr(*, "na.action")= 'omit' Named int 1098
  ..- attr(*, "names")= chr "1098"


,Departamento,Dep,Municipio,Depmun,Superficie,Poblacion,Irural,Region,Tipo,Zona,denspobl
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>
1,Antioquia,05,Alejandría,05021,128.93215,4669,48,Región Eje Cafetero,Otro,Rural,36.21284
2,Antioquia,05,Amagá,05030,84.13248,30777,28,Región Eje Cafetero,Otro,Urbano,365.81593
3,Antioquia,05,Angelópolis,05036,81.87630,5790,37,Región Eje Cafetero,Otro,Urbano,70.71643
4,Antioquia,05,Argelia,05055,245.34178,7577,46,Región Eje Cafetero,Otro,Rural,30.88345
5,Antioquia,05,Briceño,05107,376.34676,8065,52,Región Eje Cafetero,Otro,Rural,21.42971
6,Antioquia,05,Cañasgordas,05138,364.83123,15490,46,Región Eje Cafetero,Otro,Rural,42.45799


,Departamento,Dep,Municipio,Depmun,Superficie,Poblacion,Irural,Region,Tipo,Zona,denspobl
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>
306,Guaviare,95,Miraflores,95200,12808.077,6898,81,Región Llano,Otro,Rural,0.5385664
307,Vaupés,97,Caruru,97161,6368.486,3056,85,Región Llano,Otro,Rural,0.4798628
308,Vaupés,97,Pacoa,97511,13993.251,4229,91,Región Llano,Otro,Rural,0.3022171
309,Vaupés,97,Taraira,97666,6510.247,2318,97,Región Llano,Otro,Rural,0.3560541
310,Vaupés,97,Papunaua,97777,5551.154,749,95,Región Llano,Otro,Rural,0.1349269
311,Vaupés,97,Yavaraté,97889,4660.406,1067,92,Región Llano,Otro,Rural,0.2289500


Copia de Estaciones que solo considera estaciones de monitoreo localizadas como máximo a 2000 metros de altura sobre el nivel del mar, y que incluye los nombres del municipio y del departamento donde están ubicadas las estaciones.

In [17]:
e21 <- Estaciones %>%
       inner_join(Municipios,by=c("Depmun"="Depmun")) %>%
       filter(ELEV <= 2000) %>%
       select(everything(Estaciones),Municipio,Departamento,Dep)%>%
       arrange(Depmun,CODIGO) %>%
       as.data.frame()

e21a <- Estaciones %>%
        left_join(Municipios,by=c("Depmun"="Depmun")) %>%
        filter(ELEV <= 2000) %>%
        select(everything(Estaciones),Municipio,Departamento,Dep) %>%
        arrange(Depmun,CODIGO) %>%
        as.data.frame()

all.equal(e21,e21a,check.attributes=FALSE)
#all.equal(e21,E21,check.attributes=FALSE)

[1] TRUE

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cd2fca3f-dc64-4c71-8bd4-adda531fa911' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>